In [27]:
# Classifications time baby!
import classify
import pandas as pd
df = pd.read_csv('spending_predictions_months_11_to_14.csv')
df_credit = pd.read_csv("rams_batch_cur_20250325.csv")
df_filtered = df_credit[['cu_account_nbr', 'cu_crd_line']]
result = df_filtered.groupby('cu_account_nbr', as_index=False)['cu_crd_line'].max()
result.rename(columns={'cu_account_nbr': 'account_nbr'}, inplace=True)

df = df.merge(result, left_on='account_nbr', right_on='account_nbr', how='left')

upper_bound_cols = ['upper_bound_11', 'upper_bound_12', 'upper_bound_13', 'upper_bound_14']
pred = ["predicted_11","predicted_12","predicted_13","predicted_14"]
df['max_upper_bound'] = df[pred].max(axis=1)

df_eval = classify.evaluateRiskVectorized()
df_eval.rename(columns={'cu_account_nbr': 'account_nbr'}, inplace=True)
df = df.merge(df_eval, left_on='account_nbr', right_on='account_nbr', how='left')


def adjust_credit_limit(row):
    if row['max_upper_bound'] > row['history_max'] and (row['max_upper_bound']/row["cu_crd_line"]) > 0.70:  
        if row['risk'] == 'Good':
            return row['cu_crd_line'] * 2  
        elif row['risk'] == 'Risky':
            return row['cu_crd_line'] * 1.2  
        else:
            return row['cu_crd_line']  
    else:
        return row['cu_crd_line'] 

df['adjusted_credit_limit'] = df.apply(adjust_credit_limit, axis=1)

print(df[['account_nbr', 'risk', 'cu_crd_line', 'max_upper_bound', 'adjusted_credit_limit']])

df[['account_nbr', 'risk', 'cu_crd_line', 'max_upper_bound', 'adjusted_credit_limit']].to_csv('adjusted_credit_limits.csv', index=False)



           account_nbr   risk  cu_crd_line  max_upper_bound  \
0     00oyr3QppAzjLws4   Good         2900        31.577988   
1     02WRVV8yusgcZWLW   Good        15000      5681.649827   
2     033o9yHYen3xoz6k   Good        15000      3281.627696   
3     034bM166vNmgLiIA  Risky         1100       125.918843   
4     03cqv0gFjEIiQG0x  Risky         2500         0.000000   
...                ...    ...          ...              ...   
9340  zyZhjzJwhpOgSvmc  Risky         2100       907.948869   
9341  zz3nbtZXS41NZk0h   Good         2300        68.953887   
9342  zzBy2qNM78aRV580   Good        14800         0.000000   
9343  zzEuUBBmvGiVnabb   Good         6000      3078.254775   
9344  zzR9PvG7dY9u5iHU    Bad         5000       594.773186   

      adjusted_credit_limit  
0                    2900.0  
1                   15000.0  
2                   15000.0  
3                    1100.0  
4                    2500.0  
...                     ...  
9340                 2100.0  
934